About the dataset:
1. id: unique id for news article
2. title:title of news article
3. author: author of the news article
4. text: content (can be incomplete)
5. label: label which marks if article is real or fake

importing the dependencies

In [1]:
import numpy as np
import pandas as pd
import re # regular exp- searching text
from nltk.corpus import stopwords #doesnt add much value to paragraph eg a,the etc
from nltk.stem.porter import PorterStemmer# remove prefix and suffix of word
from sklearn.feature_extraction.text import TfidfVectorizer#convert text into feature vectors/numbers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
#printing the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data preprocessing

In [7]:
#loading dataset to pandas dataframe
news_dataset=pd.read_csv('/content/train.csv')

In [10]:
news_dataset.shape

(20800, 5)

In [11]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
#counting the number of missing values in dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [14]:
#replacing null values with empty string
news_dataset=news_dataset.fillna('')

In [16]:
#merging title and author column for better accuracy, can combine nything
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [17]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [18]:
#seprating the data & label
X=news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']

In [19]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:
process of reducing a word to its root word
eg: actor, actress,acting-->act(root word)

In [20]:
port_stem=PorterStemmer()

In [30]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)#^ exclusion which are not alphabets
  stemmed_conetent=stemmed_content.lower()
  stemmed_content=stemmed_content.split()# split dataset into list
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content =' '.join(stemmed_content)
  return stemmed_content

In [31]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [33]:
print(news_dataset['content'])

0        darrel lucu hous dem aid we didn even see come...
1        daniel j flynn flynn hillari clinton big woman...
2           consortiumnew com whi truth might get you fire
3        jessica purkiss civilian kill in singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t i trump poster child for...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci is said...
20798    alex ansari nato russia to hold parallel exerc...
20799                       david swanson what keep f aliv
Name: content, Length: 20800, dtype: object


In [35]:
X=news_dataset['content'].values
Y=news_dataset['label'].values

In [36]:
print(X)
print(Y)

['darrel lucu hous dem aid we didn even see comey letter until jason chaffetz tweet it'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com whi truth might get you fire' ...
 'michael j de la merc rachel abram maci is said receiv takeov approach hudson bay the new york time'
 'alex ansari nato russia to hold parallel exercis in balkan'
 'david swanson what keep f aliv']
[1 0 1 ... 0 1 1]


In [37]:
#converting textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(X)

X=vectorizer.transform(X)

In [38]:
print(X)

  (0, 16586)	0.19235766814206556
  (0, 16055)	0.29053629972079575
  (0, 15762)	0.25180682754084327
  (0, 13528)	0.22682424464275375
  (0, 8949)	0.3214177525383042
  (0, 8670)	0.25823746120670493
  (0, 7731)	0.219100353547038
  (0, 7639)	0.1611238443347535
  (0, 7040)	0.19336678366028098
  (0, 4998)	0.20625126961544935
  (0, 4028)	0.2667908047240904
  (0, 3811)	0.23915031396884368
  (0, 3619)	0.31814479104571186
  (0, 2977)	0.21821000835755877
  (0, 2501)	0.3250028762214839
  (0, 272)	0.2387684291853839
  (1, 16892)	0.30071745655510157
  (1, 6849)	0.1904660198296849
  (1, 5528)	0.7143299355715573
  (1, 3587)	0.26373768806048464
  (1, 2831)	0.19094574062359204
  (1, 2241)	0.3827320386859759
  (1, 1910)	0.15521974226349364
  (1, 1512)	0.2939891562094648
  (2, 17095)	0.29337976465513754
  :	:
  (20797, 9631)	0.17242189281191916
  (20797, 9561)	0.2918128273796141
  (20797, 9030)	0.35719284755530417
  (20797, 8404)	0.22049990081059304
  (20797, 7611)	0.13993333005918895
  (20797, 7077)	0.215

Splitting dataset into training and test data

In [39]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2, stratify=Y, random_state=2)

Training the model


In [40]:
model=LogisticRegression()

In [41]:
model.fit(X_train,Y_train)

LogisticRegression()

Evaluation

In [42]:
#accuracy_score on training data
X_train_prediction=model.predict(X_train)
training_data_accuracy= accuracy_score(X_train_prediction, Y_train)

In [43]:
print('Accuracy score of training data :',training_data_accuracy)

Accuracy score of training data : 0.9868990384615385


In [44]:
#accuracy_score on test data
X_test_prediction=model.predict(X_test)
test_data_accuracy= accuracy_score(X_test_prediction, Y_test)

In [45]:
print('Accuracy score of training data :',test_data_accuracy)

Accuracy score of training data : 0.9762019230769231


Making a predictive system

In [48]:
X_new=X_test[1]
prediction=model.predict(X_new)
print(prediction)
if(prediction[0]==0):print('News is Real')
else: print('News is Fake')

[0]
News is Real


In [49]:
print(Y_test[1])

0
